# 네이버 뉴스 데이터 수집 및 전처리 파이프라인

네이버 검색을 통해 일자별로 수집된 URL 목록을 이용해 각 언론사로 이동해 기사의 제목과 본문 데이터를 수집하고, 분석에 적합한 형태로 가공하는 전체 과정을 담고 있습니다. 데이터의 정확성과 효율성을 높이기 위해 다음과 같은 체계적인 단계를 거쳐 진행되었습니다.

In [ ]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
import re
import time
from tqdm import tqdm

# --- Selenium 라이브러리 ---
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

### 1. 날짜 폴더에 흩어져 있는 파일 병합(중복 제거 - 링크 기준)
- 목표 : 일자별 폴더에 등록되어 있는 '5개 키워드(카카오톡 개편, 카카오톡 업데이트, 카톡, 네이트온, 카카오)'로 검색한 결과 파일들을 URL 기준으로 하나의 데이터 프레임에 통합합니다.
- 핵심 : set() 자료구조를 활용하여, 여러 번 수집되었을 수 있는 중복 URL을 이 단계에서 미리 제거하여 데이터의 정합성을 확보하고, 이후의 크롤링 작업 효율을 높입니다.

In [ ]:
# --------------------------------------------------------------------------
# 1. Settings
# --------------------------------------------------------------------------
# 일자별로 수집된 검색 결과 파일 저장 경로 지정(load file)
BASE_PATH = 'E:/데일리프로젝트/251120_카카오톡 네이버기사,블로그/data'
# 결과 파일 저장 경로 지정
OUTPUT_PATH = 'E:/데일리프로젝트/251120_카카오톡 네이버기사,블로그/merged_data_news'

# 수집된 파일명(예: '251005_naver_news_links_backup1')에 키워드 대신 숫자를 입력했었기 때문에 실제 어떤 키워드를 의미하는지를 매칭시키기 
keyword_map = {
    1: '카카오톡_개편',
    2: '카카오톡_업데이트',
    3: '카톡',
    4: '네이트온',
    5: '카카오'
}

# --------------------------------------------------------------------------
# 2. 파일 경로 수집 및 키워드별 그룹화
# --------------------------------------------------------------------------
print("파일 검색 시작")

# 키워드 번호(1~5)별로 파일 경로를 저장할 딕셔너리를 생성합니다.
file_groups = {num: [] for num in range(1, 6)}

# BASE_PATH 아래 날짜 폴더가 있기 때문에 내부 폴더 검색하게 반복문 적용
for dir_name in os.listdir(BASE_PATH):
    dir_path = os.path.join(BASE_PATH, dir_name)
    
    if os.path.isdir(dir_path):
        # 폴더 내의 파일들을 확인합니다.
        for file_name in os.listdir(dir_path):
            # 'backup'로 끝나고 '.csv' 확장자를 가진 파일만 대상
            if 'backup' in file_name and file_name.endswith('.csv'):
                try:
                    # 파일명에서 숫자 부분(e.g., 'backup' -> 1) 추출
                    file_num = int(file_name.split('backup')[1].split('.csv')[0])
                    if file_num in file_groups:
                        full_path = os.path.join(dir_path, file_name)
                        file_groups[file_num].append(full_path)
                except (ValueError, IndexError):
                    # 파일명 형식이 예상과 다를 경우 오류 메시지를 출력 후 패스
                    print(f"파일명 형식이 달라 건너뜁니다: {file_name}")

print("파일 그룹화가 완료되었습니다.")
print("-" * 30)

# --------------------------------------------------------------------------
# 3. 그룹별 파일 병합, 중복 제거 및 저장
# --------------------------------------------------------------------------
# 결과 파일을 저장할 폴더가 없으면 새로 생성
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)
    print(f"'{OUTPUT_PATH}' 폴더를 생성했습니다.")

# 그룹화된 파일들을 병합 작업하기
for num, file_list in file_groups.items():
    if not file_list:
        print(f"키워드 그룹 {num} ({keyword_map[num]})에 해당하는 파일이 없습니다.")
        continue

    # 각 그룹의 모든 CSV 파일을 DataFrame으로 읽어 리스트에 담고 합치기
    df_list = [pd.read_csv(file) for file in file_list]
    
    combined_df = pd.concat(df_list, ignore_index=True)
    
    # 'URL' 컬럼을 기준으로 중복된 행 제거(첫번째 파일만 남김)
    deduplicated_df = combined_df.drop_duplicates(subset=['URL'], keep='first')
    
    # 결과 파일을 CSV로 저장(인코딩 꼭 적기!!)
    output_filename = f"combined_{num}_{keyword_map[num]}.csv"
    output_filepath = os.path.join(OUTPUT_PATH, output_filename)
    deduplicated_df.to_csv(output_filepath, index=False, encoding='utf-8-sig')
    
    # 작업 결과 출력
    print(f"'{output_filename}' 파일 생성 완료!")
    print(f" - 통합 전 총 데이터 수: {len(combined_df)}개")
    print(f" - 중복 제거 후 총 데이터 수: {len(deduplicated_df)}개")
    print("-" * 30)

print("모든 작업이 성공적으로 완료되었습니다.")

### 2. 병합된 파일에서 중요 키워드를 포함하지 않는 기사 제외하기
- 목표 : 분석과 관련 없는 데이터를 제거하여 분석의 정확도를 높입니다.
- 핵심 : 제목에 '설정한 핵심 키워드'를 포함하지 않는 뉴스 기사를 본문 수집 전에 필터링합니다.

In [ ]:
# --------------------------------------------------------------------------
# 1. Settings
# --------------------------------------------------------------------------
# 1단계 결과 폴더 연결
INPUT_PATH = 'E:/데일리프로젝트/251120_카카오톡 네이버기사,블로그/merged_data_news'

# 결과 파일 저장 경로 지정
FILTERED_OUTPUT_PATH = 'E:/데일리프로젝트/251120_카카오톡 네이버기사,블로그/filtered_data_news'

# '제목' 컬럼에 키워드가 없으면 제거
KEYWORDS_TO_CHECK = ["카카오톡", "업데이트", "개편", "카톡", "네이트온", "라인", "카카오", "메신저"]

# --------------------------------------------------------------------------
# 2. 폴더 생성 및 필터링 작업 시작
# --------------------------------------------------------------------------
# 결과 파일을 저장할 폴더가 없으면 새로 생성
if not os.path.exists(FILTERED_OUTPUT_PATH):
    os.makedirs(FILTERED_OUTPUT_PATH)
    print(f"'{FILTERED_OUTPUT_PATH}' 폴더를 생성했습니다.")

print("제목 기반 키워드 필터링을 시작합니다...")
print("-" * 30)

# --------------------------------------------------------------------------
# 3. 파일 처리 및 필터링
# --------------------------------------------------------------------------
# INPUT_PATH 폴더에 있는 모든 파일(csv 파일만 대상으로) 체크
for filename in os.listdir(INPUT_PATH):
    # .csv 파일만 대상으로 작업을 수행합니다.
    if filename.endswith('.csv'):
        file_path = os.path.join(INPUT_PATH, filename)
        
        try:
            df = pd.read_csv(file_path)
            
            original_rows = len(df)
            
            # '제목' 컬럼이 있는 지 체크
            if '제목' not in df.columns:
                print(f"'{filename}' 파일에 '제목' 컬럼이 없어 건너뜁니다.")
                continue
                
            # 키워드 리스트를 OR 조건으로 묶어 하나의 검색 패턴으로 생성
            search_pattern = '|'.join(KEYWORDS_TO_CHECK)
            
            # '제목' 컬럼에서 키워드 패턴을 포함하는 모든 행 추출
            # na=False 옵션은 '제목'이 비어있는(NaN) 경우 False로 처리하여 오류를 방지함
            filtered_df = df[df['제목'].str.contains(search_pattern, na=False)].copy()
            
            # 필터링 후 남은 데이터의 행 개수 저장
            filtered_rows = len(filtered_df)
            
            # 필터링된 결과를 새로운 파일로 저장
            output_filename = f"filtered_{filename}"
            output_filepath = os.path.join(FILTERED_OUTPUT_PATH, output_filename)
            filtered_df.to_csv(output_filepath, index=False, encoding='utf-8-sig')
            
            # 작업 결과 출력
            print(f"파일 처리 완료: {filename}")
            print(f" - 원본 데이터 수: {original_rows}개")
            print(f" - 필터링 후 데이터 수: {filtered_rows}개")
            print(f" - 결과 저장 파일: {output_filename}")
            print("-" * 30)

        except Exception as e:
            # 파일 처리 중 문제가 발생하면 오류 메시지를 출력합니다.
            print(f"'{filename}' 파일 처리 중 오류 발생: {e}")
            print("-" * 30)

print("모든 필터링 작업이 성공적으로 완료되었습니다.")

### 3. 뉴스에서 '관련 뉴스'의 언론사명이 '날짜_원문' 컬럼으로 저장되어 수정 작업 진행하기
- 목표 : 네이버 뉴스에서 '관련 뉴스'로 검색된 기사의 언론사 컬럼에 sub-article, 날짜 컬럼에 언론사명이 등록되어 각 컬럼에 맞는 데이터가 입력되도록 전처리 작업을 진행합니다. 
- 핵심 : 데이터가 밀려 입력된 컬럼 값을 재조정하여 언론사명을 기준으로 sorting & grouping 할 수 있도록 전처리 작업을 진행합니다. 

In [ ]:
# --------------------------------------------------------------------------
# 1. Settings
# --------------------------------------------------------------------------
# 2단계 결과 폴더 연결
INPUT_PATH = "E:/데일리프로젝트/251120_카카오톡 네이버기사,블로그/filtered_data_news"

# 결과 파일 저장 경로 지정
OUTPUT_PATH = "E:/데일리프로젝트/251120_카카오톡 네이버기사,블로그/press_cleaned_data_news"

# --------------------------------------------------------------------------
# 2. 메인 실행 로직: 파일 순회, 정제, 저장
# --------------------------------------------------------------------------
# 출력 폴더가 없으면 새로 생성
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)
    print(f"'{OUTPUT_PATH}' 폴더를 생성했습니다.")

# 입력 폴더에 있는 모든 파일 목록 호출
file_list = os.listdir(INPUT_PATH)

print("\n" + "="*50)
print("폴더 내 모든 .csv 파일에 대한 '언론사' 컬럼 정제 작업을 시작합니다.")
print(f"대상 폴더: {INPUT_PATH}")
print("="*50 + "\n")

# 파일 목록을 순회하며 작업을 수행합니다.
for filename in file_list:
    # .csv 파일만 대상
    if filename.endswith('.csv'):
        input_filepath = os.path.join(INPUT_PATH, filename)
        
        print(f"[{filename}] 파일 처리 중...")
        
        try:
            df = pd.read_csv(input_filepath)
                      
            # 2-1. '언론사' 컬럼이 "Sub-article"인 행들만 선택
            mask = df['언론사'] == 'Sub-article'
            
            # 2-2. 해당 행이 하나라도 있는지 확인
            if mask.sum() > 0:
                print(f" -> 'Sub-article' {mask.sum()}건 발견. 정제를 시작합니다.")

                # 2-2-1. '날짜_원문' 컬럼에서 첫 숫자가 나오기 전까지의 텍스트(언론사명)를 추출
                extracted_press = df.loc[mask, '날짜_원문'].str.extract(r'^([^\d]+)', expand=False)
                
                # 2-2-2. 추출된 언론사명으로 '언론사' 컬럼을 업데이트
                df.loc[mask, '언론사'] = extracted_press
            
            else:
                print(" -> 'Sub-article'이 없어 통과합니다.")

            # 2-3. 결과를 새로운 파일명으로 변경하여 CSV 파일로 저장
            output_filename = f"press_cleaned_{filename}"
            output_filepath = os.path.join(OUTPUT_PATH, output_filename)
            df.to_csv(output_filepath, index=False, encoding='utf-8-sig')
            
            print(f" -> 완료! '{output_filename}'으로 저장되었습니다.\n")

        except Exception as e:
            print(f" -> 오류 발생: {filename} 처리 중 문제가 발생했습니다. 오류: {e}\n")

print("모든 '언론사' 컬럼 정제 작업이 성공적으로 완료되었습니다.")

### 4. 신문사별 기사를 분리 후 기사 발행량이 많은 순으로 정렬
- 목표 : 어떤 신문사의 기사를 수집해야할지 결정하기 위해 신문사별로 기사를 정렬한 뒤, 기사 발행량순으로 정리해서 기준 설정합니다. 
- 핵심 : 기사수가 많은 신문사부터 크롤링 작업을 진행하고, 너무 기사수가 적은 신문사는 제외합니다.
<b> < 기준 설정 > 50개 이상 기사가 있는 언론사부터 먼저 크롤링 진행, 크롤링 작업 시간에 따라 3개 이상 기사가 있는 언론사까지 확장 </b>

In [ ]:
# --------------------------------------------------------------------------
# 1. Settings
# --------------------------------------------------------------------------
# 3단계 결과 폴더 연결
CLEANED_DATA_PATH = "E:/데일리프로젝트/251120_카카오톡 네이버기사,블로그/press_cleaned_data_news"

# --------------------------------------------------------------------------
# 2. 전체 통계를 위해 모든 파일을 하나로 합치기
# --------------------------------------------------------------------------
# 폴더 내의 모든 정제된 csv 파일 경로를 리스트 만들기
all_cleaned_files = [os.path.join(CLEANED_DATA_PATH, f) for f in os.listdir(CLEANED_DATA_PATH) if f.endswith('.csv')]

df_list = [pd.read_csv(file) for file in all_cleaned_files]
combined_df = pd.concat(df_list, ignore_index=True)

print("모든 정제된 뉴스 파일 통합 완료!")
print(f" -> 총 기사 수: {len(combined_df)} 개")
print("-" * 30)

# --------------------------------------------------------------------------
# 3. 언론사별 기사 수 집계 및 출력
# --------------------------------------------------------------------------
# 언론사 명에 "\n"이 추가된 경우 제거하기 (3번 작업 결과 확인 후 '토큰포스트\n' 같은 케이스가 있어서)
combined_df['언론사'] = combined_df['언론사'].str.strip()

press_counts = combined_df['언론사'].value_counts()

print("[ 언론사별 기사 수 ]")
print(press_counts)

# --------------------------------------------------------------------------
# [!] 기수 수가 몇개 이상인 언론사를 선정할지 - 체크용
# --------------------------------------------------------------------------# 
# 기사 수가 50개 이상인 언론사
print(f"50개 이상: {(press_counts >= 50).sum()} 개")
# 기사 수가 20개 이상인 언론사
print(f"20개 이상: {(press_counts >= 20).sum()} 개")
# 기사 수가 10개 이상인 언론사
print(f"10개 이상: {(press_counts >= 10).sum()} 개")

# --------------------------------------------------------------------------
# 4. 분석 대상이 될 주요 언론사 필터링
# --------------------------------------------------------------------------
# press_counts에서 기사 수가 50개 이상인 언론사만 선택
major_press = press_counts[press_counts >= 50]

print(f"총 {len(press_counts)}개 언론사 중,")
print(f"기사 수가 50개 이상인 주요 언론사는 총 {len(major_press)}개 입니다.")
print("-" * 30)
print("[ 주요 언론사 목록 (기사 수 50개 이상) ]")
print(major_press)

# --------------------------------------------------------------------------
# 5. 주요 언론사의 기사만 따로 저장 > 실제 데이터 크롤링 대상 언론사
# --------------------------------------------------------------------------
major_df = combined_df[combined_df['언론사'].isin(major_press.index)].copy()

print(f"\n주요 언론사의 총 기사 수: {len(major_df)} 개")

major_df.to_pickle("major_press_articles.pkl")
print("\n주요 언론사 기사 데이터를 'major_press_articles.pkl'로 저장했습니다.")

print(major_press.to_string())

### 5. 신문사별 본문 내용 크롤링 전 그룹핑 & 기사 날짜 및 본문 크롤링 작업
[ 그룹 1: 통신사 (News Agencies) ] > 코드 포함
- 특징: 가장 기본적인 1차 뉴스 생산자.
- 목록: 뉴스1, 뉴시스, 연합뉴스

[ 그룹 2: 주요 경제지 (Major Economic Dailies) ] > 코드 포함
- 특징: 경제/산업 분야에 집중. 
- 목록: 한국경제, 이데일리, 파이낸셜뉴스, 서울경제, 머니투데이, 매일경제, 헤럴드경제, EBN, 머니S, 이뉴스투데이, 이코노믹리뷰, 브릿지경제, 글로벌이코노믹, 비즈니스포스트, CEO스코어데일리, 파이낸셜포스트

[ 그룹 7: 기타 전문지 (Others) ] > 코드 포함
- 특징: 위 그룹에 속하지 않는 기타 전문 분야 매체.
- 목록: 중앙이코노미뉴스

================================ 샘플 코드는 그룹 1, 2, 7만 ========================================

[ 그룹 3: IT 전문 매체 (IT/Tech Media) ]
- 특징: IT, 테크, 스타트업 전문. 
- 목록: 뉴스핌, 전자신문, 디지털데일리, 조선비즈, 아이뉴스24, 디지털투데이, 지디넷코리아, 디지털타임스, 테크M, IT조선

[ 그룹 4: 주요 종합일간지 (Major General Dailies) ]
- 특징: 전통적인 주요 신문사. 
- 목록: 조선일보, 중앙일보, 세계일보, 국민일보, 동아일보, 서울신문, 메트로신문, 매일일보

[ 그룹 5: 방송사 (Broadcasting Companies) ]
- 특징: TV 방송을 기반으로 한 언론사. 
- 목록: YTN, SBS Biz, 연합뉴스TV, SBS, MBN, KBS, 한국경제TV, 서울경제TV

[ 그룹 6: 인터넷 종합/연예 매체 (Internet General/Entertainment Media) ]
- 특징: 온라인을 기반으로 한 종합/연성 뉴스. 
- 목록: 데일리안, 이투데이, 아시아경제, 노컷뉴스, 아주경제, 인사이트, 톱스타뉴스, 뉴스웨이, 뉴데일리, 포인트데일리, 천지일보, 핀포인트뉴스, 데일리한국, 서울와이어, 마이데일리

In [ ]:
def crawl_news1(url):
    content_text, article_date = "본문 수집 실패", "날짜 수집 실패"
    
    # Selenium 옵션 설정 (브라우저 창이 뜨지 않도록 headless 모드 사용)
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    
    driver = None
    try:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.get(url)
        
        # 페이지 로딩 대기
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        time.sleep(1)
        
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')
        
        # 날짜 수집
        meta_pub = soup.find('meta', property='article:published_time')
        meta_mod = soup.find('meta', property='article:modified_time')
        time_pub = soup.find('time', id='published')
        time_upd = soup.find('time', id='updated') # 추가된 부분

        if meta_pub:
            article_date = meta_pub['content']
        elif meta_mod:
            article_date = meta_mod['content']
        elif time_pub and 'datetime' in time_pub.attrs:
            article_date = time_pub['datetime']
        elif time_upd and 'datetime' in time_upd.attrs:
            article_date = time_upd['datetime']
        else:
            article_date = "날짜 태그를 찾을 수 없음"

        # 본문 수집
        article_area = soup.find('article', class_='col-lg-7')
        if article_area:
            article_body = article_area.find('div', id='articleBodyContent')
            if article_body:
                content_text = article_body.get_text(separator='\n', strip=True)
        
        if content_text == "본문 수집 실패":
             article_body_backup = soup.find('div', id='articleBodyContent')
             if article_body_backup:
                 content_text = article_body_backup.get_text(separator='\n', strip=True)

    except Exception as e:
        content_text = f"오류: {e}"
        article_date = f"오류: {e}"
    finally:
        if driver:
            driver.quit()
            
    return (content_text, article_date)

def crawl_newsis(url):
    content_text, article_date = "본문 수집 실패", "날짜 수집 실패"
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 본문 수집
            article_body = soup.find('article')
            if article_body:
                for script in article_body.find_all(['script', 'iframe', 'div'], class_=['summury', 'view_ad']):
                    script.decompose()
                content_text = article_body.get_text(separator='\n', strip=True)
            
            # 날짜 수집
            info_line = soup.find('div', class_='infoLine')
            if info_line:
                date_text = info_line.find('p', class_='txt').get_text(strip=True)
                # "등록 2025.11.21 06:00:00"에서 날짜/시간 부분만 추출 (정규표현식 사용)
                match = re.search(r'(\d{4}\.\d{2}\.\d{2}\s\d{2}:\d{2}:\d{2})', date_text)
                if match:
                    article_date = match.group(1)

    except Exception as e:
        content_text, article_date = f"오류: {e}", f"오류: {e}"
    return (content_text, article_date)

def crawl_yonhapnews(url):
    content_text, article_date = "본문 수집 실패", "날짜 수집 실패"
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            article_body = soup.find('div', class_='story-news article')
            if article_body:
                # 1. 저작권 정보를 담고 있는 마지막 p태그 제외
                paragraphs = article_body.find_all('p', class_=lambda c: c != 'txt-copyright')
                
                # 2. 각 p태그의 텍스트 리스트에 담기
                content_list = []
                for p in paragraphs:
                    text = p.get_text(strip=True)
                    if text: # 내용이 있는 텍스트만 추가
                        content_list.append(text)
                
                # 3. 리스트의 모든 텍스트를 줄바꿈 문자로 연결
                content_text = '\n'.join(content_list)
            else:
                content_text = "본문 영역(div.story-news.article) 찾기 실패"

            # 날짜 수집 - 날짜가 기사 본문 바깥에 있어서 전체 soup에서 다시 찾음
            date_tag = soup.select_one('p.txt-copyright .date .txt01')
            if date_tag:
                article_date = date_tag.get_text(strip=True).replace('송고', '').strip()
            else:
                # 날짜를 못 찾을 경우를 대비한 대체 경로
                date_tag = soup.select_one('.share-info .p-info .tit-txt')
                if date_tag:
                     article_date = date_tag.get_text(strip=True)

    except Exception as e:
        content_text, article_date = f"오류: {e}", f"오류: {e}"
        
    return (content_text, article_date)

def crawl_fnnews(url):
    content_text, article_date = "본문 수집 실패", "날짜 수집 실패"
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 본문 수집
            article_body = soup.find('div', id='article_content')
            if article_body:
                # 불필요한 영역 제거
                for tag in article_body.find_all(['div', 'p'], class_=['view_img', 'view_copyright', 'etr-keyword']):
                    tag.decompose()
                content_text = article_body.get_text(separator='\n', strip=True)
            
            # 날짜 수집
            date_area = soup.find('span', class_='row-2')
            if date_area:
                date_tag = date_area.find('p', text=re.compile('입력'))
                if date_tag:
                    # "입력 2025.11.19 07:23"에서 날짜/시간 부분만 추출
                    match = re.search(r'(\d{4}\.\d{2}\.\d{2}\s\d{2}:\d{2})', date_tag.get_text())
                    if match:
                        article_date = match.group(1)

    except Exception as e:
        content_text, article_date = f"오류: {e}", f"오류: {e}"
        
    return (content_text, article_date)

def crawl_edaily(url):
    content_text, article_date = "본문 수집 실패", "날짜 수집 실패"
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 본문 수집
            article_body = soup.find('div', class_='news_body')
            if article_body:
                # 불필요한 광고/추천 영역 제거
                for tag in article_body.find_all(['table', 'div'], class_=['gg_textshow', 'view_ad01', 'view_ad02']):
                    tag.decompose()
                content_text = article_body.get_text(separator='\n', strip=True)
            
            # 날짜 수집
            date_area = soup.find('div', class_='dates')
            if date_area:
                date_tag = date_area.find('p', text=re.compile('등록'))
                if date_tag:
                    # "등록 2025-11-17 오후 1:53:26"에서 날짜/시간 부분 추출
                    # '오후/오전'을 처리하고, 날짜 형식을 표준에 맞게 변환
                    date_text = date_tag.get_text()
                    match = re.search(r'(\d{4}-\d{2}-\d{2})\s(오전|오후)\s(\d{1,2}:\d{2}:\d{2})', date_text)
                    if match:
                        date_part, am_pm, time_part = match.groups()
                        # 오후 시간 처리 (12시간제 -> 24시간제)
                        hour = int(time_part.split(':')[0])
                        if am_pm == '오후' and hour != 12:
                            hour += 12
                        elif am_pm == '오전' and hour == 12: # 오전 12시는 0시
                            hour = 0
                        
                        time_part_24h = f"{hour:02d}:{time_part.split(':')[1]}:{time_part.split(':')[2]}"
                        article_date = f"{date_part} {time_part_24h}"

    except Exception as e:
        content_text, article_date = f"오류: {e}", f"오류: {e}"
        
    return (content_text, article_date)

def crawl_heraldcorp(url):
    content_text, article_date = "본문 수집 실패", "날짜 수집 실패"
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    driver = None
    try:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.get(url)
        
        # 본문 영역(id='articleText')이 나타날 때까지 최대 10초 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "articleText"))
        )
        
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')
        
        # 본문 수집
        article_body = soup.find('article', id='articleText')
        if article_body:
            for tag in article_body.find_all(['div', 'figure']):
                tag.decompose()
            content_text = article_body.get_text(separator='\n', strip=True)
        
        # 날짜 수집
        date_tag = soup.find('p', class_='date')
        if date_tag:
            date_text = date_tag.get_text()
            match = re.search(r'입력\s*(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2})', date_text)
            if match:
                article_date = match.group(1)

    except Exception as e:
        content_text, article_date = f"Selenium 오류: {e}", f"Selenium 오류: {e}"
    finally:
        if driver:
            driver.quit()
            
    return (content_text, article_date)

def crawl_sedaily(url):
    content_text, article_date = "본문 수집 실패", "날짜 수집 실패"
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 본문 수집
            article_body = soup.find('div', class_='article_view')
            if article_body:
                related_news = article_body.find('div', class_='art_rel')
                if related_news:
                    related_news.decompose()
                for tag in article_body.find_all(['figure', 'span'], class_=['link_figure', 'pop_link']):
                    tag.decompose()
                content_text = article_body.get_text(separator='\n', strip=True)
            
            # 날짜 수집
            date_area = soup.find('div', class_='article_info')
            if date_area:
                # article_info 영역의 모든 텍스트를 가져옴
                date_text = date_area.get_text(separator='\n', strip=True)
                # 정규 표현식으로 날짜/시간 패턴을 찾음
                match = re.search(r'(\d{4}\.\d{2}\.\d{2}\s\d{2}:\d{2}:\d{2})', date_text)
                if match:
                    article_date = match.group(1).replace('.', '-', 2).replace('.', ' ') # 형식 변경

    except Exception as e:
        content_text, article_date = f"오류: {e}", f"오류: {e}"
        
    return (content_text, article_date)

def crawl_mt(url):
    content_text, article_date = "본문 수집 실패", "날짜 수집 실패"
    
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 1. 날짜 수집
            date_tag = soup.find('span', class_='date')
            if date_tag:
                article_date = date_tag.get_text(strip=True)
            else:
                article_date = "날짜 태그(span.date) 찾기 실패"

            # 2. 본문 수집
            article_body = soup.find('div', id='articleView')
            if article_body:
                # articleView 내부의 모든 <p> 태그를 찾음
                paragraphs = article_body.find_all('p')
                
                content_list = []
                for p in paragraphs:
                    # p 태그에서 텍스트를 추출하고, 내용이 있는 경우에만 리스트에 추가
                    text = p.get_text(strip=True)
                    if text:
                        content_list.append(text)
                
                # 리스트의 텍스트들을 줄바꿈 문자로 합쳐서 최종 본문을 만듦
                content_text = '\n'.join(content_list)
            else:
                content_text = "본문 영역(div#articleView) 찾기 실패"

    except Exception as e:
        content_text, article_date = f"오류: {e}", f"오류: {e}"
        
    return (content_text, article_date)

def crawl_mk(url):
    content_text, article_date = "본문 수집 실패", "날짜 수집 실패"
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            article_body = soup.find('div', class_='news_cnt_detail_wrap')
            if article_body:
                content_text = article_body.get_text(separator='\n', strip=True)
            date_tag = soup.find('dl', class_='registration').find('dd')
            if date_tag:
                article_date = date_tag.get_text(strip=True)
    except Exception as e:
        content_text, article_date = f"오류: {e}", f"오류: {e}"
    return (content_text, article_date)

def crawl_ebn(url):
    content_text, article_date = "본문 수집 실패", "날짜 수집 실패"
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            article_body = soup.find('article', id='article-view-content-div')
            if article_body:
                content_text = article_body.get_text(separator='\n', strip=True)
            date_area = soup.find('ul', class_='article-info no-bullet')
            if date_area:
                date_tag = date_area.find('li', text=re.compile('입력'))
                if date_tag:
                    date_text = date_tag.get_text(strip=True).replace("입력", "")
                    article_date = date_text
    except Exception as e:
        content_text, article_date = f"오류: {e}", f"오류: {e}"
    return (content_text, article_date)

def crawl_hankyung(url):
    content_text, article_date = "본문 수집 실패", "날짜 수집 실패"
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 본문 수집
            article_body = soup.find('div', id='articletxt')
            if article_body:
                # 불필요한 이미지, 광고 영역 제거
                for tag in article_body.find_all(['figure', 'div'], class_=['article-figure', 'ad-area-wrap']):
                    tag.decompose()
                content_text = article_body.get_text(separator='\n', strip=True)
            
            # 날짜 수집
            date_area = soup.find('div', class_='datetime')
            if date_area:
                # '입력' 시간과 '수정' 시간 중 첫 번째(입력 시간)를 선택
                date_tag = date_area.find('span', class_='txt-date')
                if date_tag:
                    article_date = date_tag.get_text(strip=True)

    except Exception as e:
        content_text, article_date = f"오류: {e}", f"오류: {e}"
        
    return (content_text, article_date)

def crawl_moneys(url):
    content_text, article_date = "본문 수집 실패", "날짜 수집 실패"
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            article_body = soup.find('div', class_='article-body')
            if article_body:
                for tag in article_body.find_all(['table', 'script', 'center']):
                    tag.decompose()
                content_text = article_body.get_text(separator='\n', strip=True)
            date_tag = soup.find('div', class_='date').find('time')
            if date_tag and 'datetime' in date_tag.attrs:
                article_date = date_tag['datetime']
    except Exception as e:
        content_text, article_date = f"오류: {e}", f"오류: {e}"
    return (content_text, article_date)

def crawl_viva100(url):
    # 브릿지경제(viva100)
    content_text, article_date = "본문 수집 실패", "날짜 수집 실패"
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            article_body = soup.find('div', class_='news_content')
            if article_body:
                for tag in article_body.find_all(['figure', 'div', 'script']):
                    tag.decompose()
                content_text = article_body.get_text(separator='\n', strip=True)
            date_area = soup.find('div', class_='news_date')
            if date_area:
                date_tag = date_area.find('span', text=re.compile('배포일'))
                if date_tag:
                    date_text = date_tag.get_text(strip=True)
                    match = re.search(r'(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2})', date_text)
                    if match:
                        article_date = match.group(1)
    except Exception as e:
        content_text, article_date = f"오류: {e}", f"오류: {e}"
    return (content_text, article_date)

def crawl_genews(url):
    # 글로벌이코노믹
    content_text, article_date = "본문 수집 실패", "날짜 수집 실패"
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            article_body = soup.find('div', class_='vtxt detailCont')
            if article_body:
                for tag in article_body.find_all(['div', 'figure', 'script', 'ins']):
                    tag.decompose()
                content_text = article_body.get_text(separator='\n', strip=True)
            
            date_area = soup.find('p', class_='r3')
            if date_area:
                # div.e1 영역의 텍스트를 통째로 가져온다
                date_text_full = date_area.get_text(strip=True)
                # 그 안에서 '입력' 다음에 오는 날짜 패턴을 정규식으로 찾는다
                match = re.search(r'입력(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2})', date_text_full)
                if match:
                    article_date = match.group(1)
                
    except Exception as e:
        content_text, article_date = f"오류: {e}", f"오류: {e}"
    return (content_text, article_date)

def crawl_businesspost(url):
    content_text, article_date = "본문 수집 실패", "날짜 수집 실패"
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 본문 수집
            article_body = soup.find('div', class_='detail_editor')
            if article_body:
                # 불필요한 광고, 스크립트, 저작권 영역 제거
                # 마지막 저작권 문구 제거
                copyright_tag = article_body.find('div', class_='copyright')
                if copyright_tag:
                    copyright_tag.decompose()
                content_text = article_body.get_text(separator='\n', strip=True)

            # 날짜 수집
            date_area = soup.find('div', class_='author_info')
            if date_area:
                # author_info div 안에 있는 span 태그들을 모두 찾아서 마지막 것을 선택
                all_spans = date_area.find_all('span')
                if all_spans:
                    date_tag = all_spans[-1]
                    article_date = date_tag.get_text(strip=True)
                    
    except Exception as e:
        content_text, article_date = f"오류: {e}", f"오류: {e}"
        
    return (content_text, article_date)

def crawl_ceoscore(url):
    content_text, article_date = "본문 수집 실패", "날짜 수집 실패"
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            article_body = soup.find('div', class_='article_content')
            if article_body:
                for tag in article_body.find_all(['h3', 'div', 'iframe', 'script']):
                    tag.decompose()
                content_text = article_body.get_text(separator='\n', strip=True)
            date_area = soup.find('div', class_='date')
            if date_area:
                date_text = date_area.get_text()
                match = re.search(r'입력\s*(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2})', date_text)
                if match:
                    article_date = match.group(1)
    except Exception as e:
        content_text, article_date = f"오류: {e}", f"오류: {e}"
    return (content_text, article_date)

def crawl_common_type_j(url):
    # 공통 함수 - 요기 포함되는 케이스가 제법 있었음
    content_text, article_date = "본문 수집 실패", "날짜 수집 실패"
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 본문 수집
            article_body = soup.find('article', id='article-view-content-div')
            if article_body:
                for tag in article_body.find_all(['figure', 'div', 'style', 'script']):
                    tag.decompose()
                content_text = article_body.get_text(separator='\n', strip=True)
            
            # 날짜 수집
            # 1순위: <div class="info-group">
            date_area = soup.find('div', class_='info-group')
            if not date_area:
                # 2순위: <ul class="infomation">
                date_area = soup.find('ul', class_='infomation')
            
            if date_area:
                # '입력' 또는 '승인' 텍스트를 포함하는 li 태그를 찾음
                for li in date_area.find_all('li'):
                    li_text = li.get_text()
                    if '입력' in li_text or '승인' in li_text:
                        match = re.search(r'(\d{4}[-.]\d{2}[-.]\d{2}\s\d{2}:\d{2})', li_text)
                        if match:
                            article_date = match.group(1).replace('.', '-')
                            break # 날짜 찾으면 중단
                            
    except Exception as e:
        content_text, article_date = f"오류: {e}", f"오류: {e}"
    return (content_text, article_date)

In [ ]:
# --------------------------------------------------------------------------
# 1. 마스터 컨트롤러 및 설정
# --------------------------------------------------------------------------
# 언론사와 함수 매칭
crawler_map = {
    "뉴스1": crawl_news1, 
    "뉴시스": crawl_newsis,
    "연합뉴스": crawl_yonhapnews,
    "파이낸셜뉴스": crawl_fnnews, 
    "이데일리": crawl_edaily, 
    "헤럴드경제": crawl_heraldcorp,
    "한국경제": crawl_hankyung,
    "뉴스핌": crawl_newspim, 
    "전자신문": crawl_etnews, 
    "디지털데일리": crawl_ddaily,
    "조선일보": crawl_chosun, 
    "중앙일보": crawl_joongang, 
    "세계일보": crawl_segye,
    "YTN": crawl_ytn, 
    "SBS Biz": crawl_sbsbiz, 
    "연합뉴스TV": crawl_yonhapnewstv,
    "SBS": crawl_sbs, 
    "연합뉴스": crawl_yonhapnews,
    "서울경제": crawl_sedaily,
    "머니투데이": crawl_mt,
    "매일경제": crawl_mk,
    "EBN": crawl_ebn,
    "머니S": crawl_moneys,
    "이뉴스투데이": crawl_common_type_j,
    "이코노믹리뷰": crawl_common_type_j,
    "브릿지경제": crawl_viva100,
    "CEO스코어데일리": crawl_ceoscore,
    "파이낸셜포스트": crawl_common_type_j,
    "글로벌이코노믹": crawl_genews,
    "비즈니스포스트": crawl_businesspost,
    "조선비즈": crawl_chosunbiz_selenium,
    "아이뉴스24": crawl_inews24,
    "디지털투데이": crawl_common_type_j, 
    "지디넷코리아": crawl_zdnet,
    "테크M": crawl_techm, 
    "IT조선": crawl_common_type_j,
    "디지털타임스": crawl_dt,  
    "국민일보": crawl_kmib,
    "동아일보": crawl_donga,
    "서울신문": crawl_seoul,
    "메트로신문": crawl_metroseoul,
    "매일일보": crawl_mi, 
    "KBS": crawl_kbs,
    "한국경제TV": crawl_wowtv,
    "MBN": crawl_mbn,
    "서울경제TV": crawl_sentv,
    "데일리안": crawl_dailian,
    "이투데이": crawl_etoday,
    "아시아경제": crawl_asiae,
    "노컷뉴스": crawl_nocutnews,     
    "아주경제": crawl_ajunews,
    "인사이트": crawl_insight,
    "톱스타뉴스": crawl_topstarnews,
    "뉴스웨이": crawl_newsway,
    "뉴데일리": crawl_newdaily,
    "포인트데일리": crawl_common_type_j,
    "천지일보": crawl_common_type_j,
    "핀포인트뉴스": crawl_common_type_j,
    "데일리한국": crawl_common_type_j,
    "서울와이어": crawl_common_type_j,
    "마이데일리": crawl_mydaily,
    "중앙이코노미뉴스": crawl_common_type_j    
}

# [중간 저장 주기 설정]
SAVE_INTERVAL = 100 # 100개마다 저장

def get_content_and_date(row):
    """'언론사'에 맞는 크롤링 함수를 호출하고 (본문, 날짜) 튜플을 반환"""
    press = row['언론사']
    url = row['URL']
    
    crawler_function = crawler_map2.get(press)
    
    if crawler_function:
        try:
            # 0.5초 대기 후 함수 실행 (서버 부하 감소)
            time.sleep(0.5)
            return crawler_function(url)
        except Exception as e:
            return (f"컨트롤러 오류: {e}", f"컨트롤러 오류: {e}")
    else:
        return ("해당 언론사 크롤러 없음", "해당 언론사 크롤러 없음")

# --------------------------------------------------------------------------
# 2. 크롤링 실행 (apply, 중간 저장 및 이어하기 로직)
# --------------------------------------------------------------------------
# 출력 파일 경로 설정
output_filename = "crawled_press_phase2.pkl"

# --- 이어하기 로직 ---
if os.path.exists(output_filename):
    print(f"'{output_filename}' 파일이 존재합니다. 작업을 이어서 시작합니다.")
    df_phase2 = pd.read_pickle(output_filename)
else:
    print("새로운 작업을 시작합니다.")
    df_phase2 = combined_df[combined_df['언론사'].isin(press_list_phase2)].copy()
    # '본문'과 '기사작성일' 컬럼이 없으면 새로 만들고 비어있음(None)으로 초기화
    if '본문' not in df_phase2.columns:
        df_phase2['본문'] = None
    if '기사작성일' not in df_phase2.columns:
        df_phase2['기사작성일'] = None

# 크롤링이 아직 되지 않은 행들만 대상으로 작업
to_crawl = df_phase2[df_phase2['본문'].isnull()]

if to_crawl.empty:
    print("모든 기사 수집이 이미 완료되었습니다.")
else:
    print("="*60)
    print("★★★ 2단계 크롤링을 시작합니다 (대상: 17개 언론사) ★★★")
    print(f"총 {len(df_phase2)}개 중 {len(to_crawl)}개의 남은 기사를 수집합니다. (PC를 켜두세요)")
    print("="*60 + "\n")

    # --- apply와 tqdm을 함께 사용 ---
    results = []
    # tqdm으로 남은 작업량에 대한 진행률 바를 생성
    for index, row in tqdm(to_crawl.iterrows(), total=len(to_crawl)):
        # 각 행에 대해 마스터 함수를 실행
        result_tuple = get_content_and_date(row)
        results.append(result_tuple)
        
        # --- 중간 저장 로직 ---
        current_progress_index = len(results)
        if current_progress_index % SAVE_INTERVAL == 0:
            # 지금까지 수집된 결과를 데이터프레임에 임시로 업데이트
            temp_df = pd.DataFrame(results, index=to_crawl.index[:current_progress_index], columns=['본문', '기사작성일'])
            df_phase2.update(temp_df)
            # 중간 저장
            df_phase2.to_pickle(output_filename)
            tqdm.write(f" -> 중간 저장 완료 ({current_progress_index}/{len(to_crawl)} 진행)")

    # 모든 크롤링이 끝난 후, 수집된 전체 결과를 데이터프레임에 최종 업데이트
    final_df = pd.DataFrame(results, index=to_crawl.index, columns=['본문', '기사작성일'])
    df_phase2.update(final_df)
    
    print("\n★★★ 1단계 크롤링이 모두 완료되었습니다! ★★★")

    # 최종 결과 저장
    df_phase2.to_pickle(output_filename)
    print(f"\n1단계 크롤링 최종 결과를 '{output_filename}' 파일로 저장했습니다.")

df_phase2 = pd.read_pickle('crawled_press_phase2.pkl')

# --------------------------------------------------------------------------
# 3. 결과 확인
# --------------------------------------------------------------------------

# 기본 정보 확인
df_phase2.head(5)
df_phase2.shape
df_phase2.info()

# 날짜 컬럼에 '실패'나 '못했습니다' 같은 에러 메시지가 포함된 행 찾기
error_mask = df_phase2['기사작성일'].astype(str).str.contains('실패|못했습니다', na=False)
error_df = df_phase2[error_mask]

# 통계 출력
print(f"전체 수집 데이터: {len(df_phase2)}건")
print(f"날짜 수집 실패 데이터: {len(error_df)}건")
print("-" * 30)
print("[ 언론사별 날짜 수집 실패 현황 ]")
print(error_df['언론사'].value_counts())

### 6. 검색 키워드별 그룹핑을 위해 키워드 컬럼 추가
목표 : 분석 작업시 전체 분석 외 세부 분석을 진행할 수 있도록 검색 키워드 컬럼을 추가합니다. 
핵심 : 세부 분석은 언론사 컬럼과 키워드 컬럼을 이용할 예정입니다. 

In [ ]:
# 각 기사들이 어떤 키워드로 조회되었던 내용인지 항목 추가 필요
# --------------------------------------------------------------------------
# 1. setting
# --------------------------------------------------------------------------
# 5번 작업 결과 폴더 추가
INPUT_PATH = "E:/데일리프로젝트/251120_카카오톡 네이버기사,블로그/press_cleaned_data_news" 

# 5번 작업 결과 파일명 추가
PKL_FILE_PATH = "crawled_press_phase2.pkl"

# --------------------------------------------------------------------------
# 2. 원본 CSV 파일들을 이용해 [URL - 검색키워드] 족보 만들기
# --------------------------------------------------------------------------
print("원본 파일에서 키워드 정보를 추출합니다...")
url_keyword_map = []

if os.path.exists(INPUT_PATH):
    for filename in os.listdir(INPUT_PATH):
        if filename.endswith('.csv'):
            # 파일명에서 키워드 추출
            try:
                keyword = filename.split('_', 5)[-1].replace('.csv', '')
            except:
                keyword = filename 

            temp_df = pd.read_csv(os.path.join(INPUT_PATH, filename))
            for url in temp_df['URL']:
                url_keyword_map.append({'URL': url, '검색키워드': keyword})
else:
    print(f"오류: 입력 폴더 경로를 찾을 수 없습니다: {INPUT_PATH}")

keyword_df = pd.DataFrame(url_keyword_map).drop_duplicates(subset=['URL'])
print(f" -> 총 {len(keyword_df)}개의 URL에 대한 키워드 족보 확보 완료.")

# --------------------------------------------------------------------------
# 3. 기존 크롤링 결과(pkl) 불러오기 및 병합
# --------------------------------------------------------------------------
if os.path.exists(PKL_FILE_PATH):
    df_main = pd.read_pickle(PKL_FILE_PATH)
    print(f"기존 데이터 {len(df_main)}건 로드 완료.")
    
    # 기존 데이터에 '검색키워드' 컬럼 추가 (Left Join)
    df_main = pd.merge(df_main, keyword_df, on='URL', how='left')
    print(" -> 기존 데이터에 검색 키워드 정보 결합 완료!")
else:
    print("오류: 크롤링 결과 파일(pkl)이 없습니다.")

# --------------------------------------------------------------------------
# 4. 최종 결과 저장
# --------------------------------------------------------------------------
# 덮어쓰지 않고 새 이름(_v2)으로 저장
df_main.to_pickle("crawled_press_phase2_v2.pkl")
print("\n최종 데이터 저장 완료: crawled_press_phase2_v2.pkl")

# 확인
display(df_main.head())

### 7. 파일별 자연어 처리 작업 진행
목표 : 자연어 분석을 위해서 수집된 블로그 '제목과 본문'에서 특문, 이모티콘, 자음과 모음만 있는 경우를 제외하는 전처리 작업 진행합니다.
핵심 : 숫자와 영문은 제거되지 않도록 처리가 필요합니다.

In [ ]:
import re
import emoji

# 자연어 분석을 위한 함수 만들기 & 테스트
def clean_text_with_lib(text):
    text = str(text)
    
    # 1. [수정됨] emoji 라이브러리를 사용하여 모든 이모티콘을 찾아 빈 문자열로 바꾸기
    text = emoji.replace_emoji(text, replace='')
    
    # 2. 한글, 영어, 숫자를 제외한 특수문자 제거
    text = re.sub(r'[^A-Za-z0-9가-힣ㄱ-ㅎㅏ-ㅣ\s]', '', text)
    
    # 3. 자음 또는 모음만 반복되는 경우 제거 (ㅋㅋㅋ, ㅜㅜㅜ)
    text = re.sub(r'([ㄱ-ㅎㅏ-ㅣ])\1{2,}', '', text)
    
    # 4. 여러 개의 공백을 하나의 공백으로 변경
    text = re.sub(r'\s+', ' ', text)
    
    # 5. 앞뒤 공백 제거
    text = text.strip()
    return text

# 테스트 코드 -> 결과 확인 후 반영
test_review1 = "업데이트 최악 1점 드립니다.ㅠㅠㅠ UI 돌려놔!!!! #카카오"
test_review2 = "ㅋㅋㅋ Only salvation is Jesus, believe in Jesus 🙏"
test_review3 = "이모티콘😄은 좋은데, 앱 속도가 너무 느려요..."
test_review4 = "0점이 없어서 1점남김ㅋㅋ"
print(f"원본: {test_review1} -> 정제 후: {clean_text_with_lib(test_review1)}")
print(f"원본: {test_review2} -> 정제 후: {clean_text_with_lib(test_review2)}")
print(f"원본: {test_review3} -> 정제 후: {clean_text_with_lib(test_review3)}")
print(f"원본: {test_review4} -> 정제 후: {clean_text_with_lib(test_review4)}")

# 적용할 pkl 파일 호출
df = pd.read_pickle("final_crawled_news_all.pkl")

df["제목_cleaned"] = df["제목"].apply(clean_text_with_lib)
df["본문_cleaned"] = df["본문"].apply(clean_text_with_lib)

# 최종 통합본 저장
df.to_pickle("final_crawled_cleande_news_all.pkl")
print("\n최종 통합 데이터를 'final_crawled_cleande_news_all.pkl' 파일로 저장했습니다.")